In [1]:
root_dir = "../../Downloads/GENOME540/"
protein = "ferritins" # "bacterioferritins" "transthyretins" "hemoglobins" "insulins"
fastas_path = root_dir + protein + ".txt"
score_matrix_path = root_dir + "blosum62.txt"
output_path = root_dir + "_wdag.txt"

In [2]:
def read_fastas(filename):
    fastas = {}
    with open(filename, 'r') as f:
        lines = f.read().split('>')
        lines = [line for line in lines if line]
        for line in lines:
            title, seq = line.split('\n', 1)
            fastas[title] = seq.replace('\n', '')
    fastas = list(fastas.values())
    return fastas, list(set("".join(fastas))) + ["-"]

fastas, aminos = read_fastas(fastas_path)
fastas

['MALAPSKVSTFSGFSPKPSVGGAQKNPTCSVSLSFLNEKLGSRNLRVCASTVPLTGVIFEPFEEVKKSELAVPTAPQVSLARQNYADECESAINEQINVEYNASYVYHSLFAYFDRDNVALKGFAKFFKESSEEEREHAEKLMKYQNTRGGRVVLHPIKNAPSEFEHVEKGDALYAMELALSLEKLVNEKLLNVHSVADRNNDPQMADFIESEFLSEQVESIKKISEYVAQLRRVGKGHGVWHFDQRLLD',
 'MLLKAAPAFALLNTQGENLSPLFSSSKSFSPKNGNRFVVSASKATNHKPLTGVVFEPFEELKKELMLVPAVPDTSLCRQKYSDDCEAAINEQINVEYNNSYVYHAMFAYFDRDNVALKGLAKFFKESSLEEREHAEKLMEFQNKRGGRVKLLSICAPPTEFDHCEKGDALYAMELALCLEKLTNQRLLNLHAVASRSNDVHLADFLESEFLVEQVDAIKKISEYVAQLRRVGQGHGVWQFDQMLLNEGAAA',
 'MMLRVSPSPAAAVPTQLSGAPATPAPVVRVAAPRGVASPSAGAACRAAGKGKEVLSGVVFQPFEEIKGELALVPQSPDKSLARHKFVDDCEAALNEQINVEYNASYAYHSLFAYFDRDNVALKGFAKFFKESSDEEREHAEKLMEYQNKRGGRVRLQSIVTPLTEFDHPEKGDALYAMELALALEKLVNEKLHNLHGVATRCNDPQLTDFIESEFLEEQGEAINKISKYVAQLRRVGKGHGVWHFDQMLLEEEA']

In [3]:
def read_score_matrix(filename):
    with open(filename, 'r') as f:
        amino_acids = f.readline().strip().split()
        score_matrix = {}
        for line in f:
            scores = line.strip().split()
            amino_acid = scores[0]
            score_matrix[amino_acid] = {}
            for i in range(len(amino_acids)):
                score_matrix[amino_acid][amino_acids[i]] = int(scores[i + 1])
    return score_matrix

blosum62 = read_score_matrix(score_matrix_path)
gap_pen = -6

In [4]:
def get_weight(edge, score_matrix, gap_penalty):
    x1, x2, x3 = edge
    pair_weights = [0, 0, 0]
    if x1 != "-" and x2 != "-":
        pair_weights[0] = score_matrix[x1][x2]
    elif x1 != "-":
        pair_weights[0] = gap_penalty
    elif x2 != "-":
        pair_weights[0] = gap_penalty
    if x1 != "-" and x3 != "-":
        pair_weights[1] = score_matrix[x1][x3]
    elif x1 != "-":
        pair_weights[1] = gap_penalty
    elif x3 != "-":
        pair_weights[1] = gap_penalty
    if x2 != "-" and x3 != "-":
        pair_weights[2] = score_matrix[x2][x3]
    elif x2 != "-":
        pair_weights[2] = gap_penalty
    elif x3 != "-":
        pair_weights[2] = gap_penalty
    return sum(pair_weights)

In [5]:
import itertools

def display_weights(letters, score_matrix, gap_penalty):
    edges = sorted(["".join(x) for x in list(itertools.product(letters, repeat=3))])[1:]
    print("Edge weights:")
    for edge in edges:
        print(edge, get_weight(edge, score_matrix, gap_penalty))

display_weights(aminos, blosum62, gap_pen)

Edge weights:
--A -12
--C -12
--D -12
--E -12
--F -12
--G -12
--H -12
--I -12
--K -12
--L -12
--M -12
--N -12
--P -12
--Q -12
--R -12
--S -12
--T -12
--V -12
--W -12
--Y -12
-A- -12
-AA -8
-AC -12
-AD -14
-AE -13
-AF -14
-AG -12
-AH -14
-AI -13
-AK -13
-AL -13
-AM -13
-AN -14
-AP -13
-AQ -13
-AR -13
-AS -11
-AT -12
-AV -12
-AW -15
-AY -14
-C- -12
-CA -12
-CC -3
-CD -15
-CE -16
-CF -14
-CG -15
-CH -15
-CI -13
-CK -15
-CL -13
-CM -13
-CN -15
-CP -15
-CQ -15
-CR -15
-CS -13
-CT -13
-CV -13
-CW -14
-CY -14
-D- -12
-DA -14
-DC -15
-DD -6
-DE -10
-DF -15
-DG -13
-DH -13
-DI -15
-DK -13
-DL -16
-DM -15
-DN -11
-DP -13
-DQ -12
-DR -14
-DS -12
-DT -13
-DV -15
-DW -16
-DY -15
-E- -12
-EA -13
-EC -16
-ED -10
-EE -7
-EF -15
-EG -14
-EH -12
-EI -15
-EK -11
-EL -15
-EM -14
-EN -12
-EP -13
-EQ -10
-ER -12
-ES -12
-ET -13
-EV -14
-EW -15
-EY -14
-F- -12
-FA -14
-FC -14
-FD -15
-FE -15
-FF -6
-FG -15
-FH -13
-FI -12
-FK -15
-FL -12
-FM -12
-FN -15
-FP -16
-FQ -15
-FR -15
-FS -14
-FT -14
-FV -13
-FW -11

In [6]:
class Vertex:
    def __init__(self, label):
        self.label = label
        self.edges = []

    def add_edge(self, edge):
        self.edges.append(edge)

    def sort_edges(self):
        self.edges = sorted(self.edges, key=lambda edge: -edge.weight)

class Edge:
    def __init__(self, label, start, end, weight):
        self.label = label
        self.start = start
        self.end = end
        self.weight = weight

    def __lt__(self, other):
        return self.weight < other.weight

In [7]:
def create_graph(sequences, score_matrix, gap_penalty):
    seq1, seq2, seq3 = sequences[0], sequences[1], sequences[2]
    n1, n2, n3 = len(seq1), len(seq2), len(seq3)

    vertices = [[[Vertex(f"{i}_{j}_{k}") for k in range(n3 + 1)] for j in range(n2 + 1)] for i in range(n1 + 1)]

    for i in range(n1 + 1):
        for j in range(n2 + 1):
            for k in range(n3 + 1):
                vertex = vertices[i][j][k]

                if i < n1:
                    edge_label = seq1[i] + "-" + "-"
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i + 1][j][k]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if j < n2:
                    edge_label = "-" + seq2[j] + "-"
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i][j + 1][k]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if k < n3:
                    edge_label = "-" + "-" + seq3[k]
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i][j][k + 1]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if i < n1 and j < n2:
                    edge_label = seq1[i] + seq2[j] + "-"
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i + 1][j + 1][k]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if j < n2 and k < n3:
                    edge_label = "-" + seq2[j] + seq3[k]
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i][j + 1][k + 1]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if i < n1 and k < n3:
                    edge_label = seq1[i] + "-" + seq3[k]
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i + 1][j][k + 1]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

                if i < n1 and j < n2 and k < n3:
                    edge_label = seq1[i] + seq2[j] + seq3[k]
                    weight = get_weight(edge_label, score_matrix, gap_penalty)
                    neighbor = vertices[i + 1][j + 1][k + 1]
                    edge = Edge(edge_label, vertex, neighbor, weight)
                    vertex.add_edge(edge)

    return [vertex for row in vertices for col in row for vertex in col]

verts = create_graph(fastas, blosum62, gap_pen)

In [8]:
def write_graph(vertices, filename, write=False):
    if write:
        with open(filename, "w") as f:
            for vertex in vertices:
                f.write(f"V {vertex.label}\n")
                for edge in vertex.edges:
                    f.write(f"E {edge.start.label} {edge.end.label} {edge.weight}\n")

write_graph(verts, output_path)

In [9]:
from collections import Counter

print("Edge counts:")
counts = sorted(Counter([y.label for x in verts for y in x.edges]).items())
for count in counts:
    print(count[0], count[1])

Edge counts:
--A 2024064
--C 189756
--D 695772
--E 1644552
--F 759024
--G 1012032
--H 506016
--I 379512
--K 1012032
--L 1518048
--M 316260
--N 569268
--P 885528
--Q 632520
--R 822276
--S 948780
--T 379512
--V 1265040
--W 63252
--Y 442764
-A- 1664130
-AA 208832
-AC 19578
-AD 71786
-AE 169676
-AF 78312
-AG 104416
-AH 52208
-AI 39156
-AK 104416
-AL 156624
-AM 32630
-AN 58734
-AP 91364
-AQ 65260
-AR 84838
-AS 97890
-AT 39156
-AV 130520
-AW 6526
-AY 45682
-C- 320025
-CA 40160
-CC 3765
-CD 13805
-CE 32630
-CF 15060
-CG 20080
-CH 10040
-CI 7530
-CK 20080
-CL 30120
-CM 6275
-CN 11295
-CP 17570
-CQ 12550
-CR 16315
-CS 18825
-CT 7530
-CV 25100
-CW 1255
-CY 8785
-D- 704055
-DA 88352
-DC 8283
-DD 30371
-DE 71786
-DF 33132
-DG 44176
-DH 22088
-DI 16566
-DK 44176
-DL 66264
-DM 13805
-DN 24849
-DP 38654
-DQ 27610
-DR 35893
-DS 41415
-DT 16566
-DV 55220
-DW 2761
-DY 19327
-E- 1472115
-EA 184736
-EC 17319
-ED 63503
-EE 150098
-EF 69276
-EG 92368
-EH 46184
-EI 34638
-EK 92368
-EL 138552
-EM 28865
-EN 51

In [10]:
from collections import deque

def topological_sort(vertices):
    indegree = {v: 0 for v in vertices}
    for v in vertices:
        for e in v.edges:
            indegree[e.end] += 1
    
    queue = deque(v for v in vertices if indegree[v] == 0)
    
    result = []
    while queue:
        v = queue.popleft()
        result.append(v)
        for e in v.edges:
            indegree[e.end] -= 1
            if indegree[e.end] == 0:
                queue.append(e.end)
    
    if len(result) != len(vertices):
        raise RuntimeError("bad graph")
    
    return result

def get_path(vertex, longest_paths):
    path = []
    while longest_paths[vertex] != 0:
        max_edge = max(vertex.edges, key=lambda e: longest_paths[e.end] + e.weight)
        path.append(max_edge)
        vertex = max_edge.end
    return path[::-1]

def highest_weight_path(vertices):

    sorted_vertices = topological_sort(vertices)
    
    longest_paths = {v: 0 for v in vertices}
    
    for v in reversed(sorted_vertices):
        longest_path = 0
        for e in v.edges:
            longest_path = max(longest_path, longest_paths[e.end] + e.weight)
        longest_paths[v] = longest_path
    
    highest_weight = max(longest_paths.values())
    highest_path = []
    for v in vertices:
        if longest_paths[v] == highest_weight:
            highest_path = get_path(v, longest_paths)
            break
    
    if highest_path:
        return highest_weight, [x.label for x in highest_path][::-1]
    else:
        return 0, []

score, path = highest_weight_path(verts)

print("Score:", score)
print("Local alignment:")
print("\n".join(path))

Score: 2325
Local alignment:
PPV
LLL
TTS
GGG
VVV
IVV
FFF
EEQ
PPP
FFF
EEE
EEE
VLI
KKK
KKG
S--
EEE
LLL
AMA
-LL
VVV
PPP
TAQ
AVS
PPP
QDD
VTK
SSS
LLL
ACA
RRR
QQH
NKK
YYF
ASV
DDD
EDD
CCC
EEE
SAA
AAA
IIL
NNN
EEE
QQQ
III
NNN
VVV
EEE
YYY
NNN
ANA
SSS
YYY
VVA
YYY
HHH
SAS
LML
FFF
AAA
YYY
FFF
DDD
RRR
DDD
NNN
VVV
AAA
LLL
KKK
GGG
FLF
AAA
KKK
FFF
FFF
KKK
EEE
SSS
SSS
ELD
EEE
EEE
RRR
EEE
HHH
AAA
EEE
KKK
LLL
MMM
KEE
YFY
QQQ
NNN
TKK
RRR
GGG
GGG
RRR
VVV
VKR
LLL
HLQ
PSS
III
KCV
NAT
APP
PPL
STT
EEE
FFF
EDD
HHH
VCP
EEE
KKK
GGG
DDD
AAA
LLL
YYY
AAA
MMM
EEE
LLL
AAA
LLL
SCA
LLL
EEE
KKK
LLL
VTV
NNN
EQE
KRK
LLL
LLH
NNN
VLL
HHH
SAG
VVV
AAA
DST
RRR
NSC
NNN
DDD
PVP
QHQ
MLL
AAT
DDD
FFF
ILI
EEE
SSS
EEE
FFF
LLL
SVE
EEE
QQQ
VVG
EDE
SAA
III
KKN
KKK
III
SSS
EEK
YYY
VVV
AAA
QQQ
LLL
RRR
RRR
VVV
GGG
KQK
GGG
HHH
GGG
VVV
WWW
HQH
FFF
DDD
QQQ
RMM
LLL
LLL
DNE


In [11]:
def extract_alignments(alignment):
    alignment1 = "".join([x[0] for x in alignment if x[0] != '-'])
    alignment2 = "".join([x[1] for x in alignment if x[1] != '-'])
    alignment3 = "".join([x[2] for x in alignment if x[2] != '-'])
    return [alignment1, alignment2, alignment3]

aligns = extract_alignments(path)

In [12]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def highlight_substring(full_string, substring):

    start_index = full_string.find(substring)
    if start_index == -1:
        return full_string
    end_index = start_index + len(substring)

    print(f"{full_string[:start_index]}{bcolors.OKGREEN}{full_string[start_index:end_index]}{bcolors.ENDC}{full_string[end_index:]}")

def highlight_alignments(sequences, alignments):

    for s, a in zip(sequences, alignments):
        highlight_substring(s, a)

highlight_alignments(fastas, aligns)

MALAPSKVSTFSGFSPKPSVGGAQKNPTCSVSLSFLNEKLGSRNLRVCASTVPLTGVIFEPFEEVKKSELAVPTAPQVSLARQNYADECESAINEQINVEYNASYVYHSLFAYFDRDNVALKGFAKFFKESSEEEREHAEKLMKYQNTRGGRVVLHPIKNAPSEFEHVEKGDALYAMELALSLEKLVNEKLLNVHSVADRNNDPQMADFIESEFLSEQVESIKKISEYVAQLRRVGKGHGVWHFDQRLLD
MLLKAAPAFALLNTQGENLSPLFSSSKSFSPKNGNRFVVSASKATNHKPLTGVVFEPFEELKKELMLVPAVPDTSLCRQKYSDDCEAAINEQINVEYNNSYVYHAMFAYFDRDNVALKGLAKFFKESSLEEREHAEKLMEFQNKRGGRVKLLSICAPPTEFDHCEKGDALYAMELALCLEKLTNQRLLNLHAVASRSNDVHLADFLESEFLVEQVDAIKKISEYVAQLRRVGQGHGVWQFDQMLLNEGAAA
MMLRVSPSPAAAVPTQLSGAPATPAPVVRVAAPRGVASPSAGAACRAAGKGKEVLSGVVFQPFEEIKGELALVPQSPDKSLARHKFVDDCEAALNEQINVEYNASYAYHSLFAYFDRDNVALKGFAKFFKESSDEEREHAEKLMEYQNKRGGRVRLQSIVTPLTEFDHPEKGDALYAMELALALEKLVNEKLHNLHGVATRCNDPQLTDFIESEFLEEQGEAINKISKYVAQLRRVGKGHGVWHFDQMLLEEEA
